# Description
This notebook reparametrizes the asymmetric logistic function in various ways using `sympy`.

In [1]:
import numpy
from matplotlib import pyplot
import ipywidgets
import sympy
from IPython.display import display, Latex
import calibr8
import models

In [2]:
cm_glucose_linear = models.get_glucose_model_linear()
cm_glucose = models.get_glucose_model()
cm_biomass = models.get_biomass_model()

In [3]:
def sympyfied_predict(cm):
    theta = [
        *sympy.symbols(" ".join(cm.theta_names[:-1]), real=True),
        *[sympy.symbols(" ".join(cm.theta_names[-1:]), real=True, positive=True),]
    ]
    x = sympy.symbols("x")

    # Replace numpy function that are not sympy-compatible
    # with sympy-compatible versions.
    calibr8.core.numpy.exp = sympy.exp
    calibr8.core.numpy.array = lambda x: x
    calibr8.core.numpy.log10 = lambda x: sympy.log(x, 10)
    params = cm.predict_dependent(x, theta=theta)
    return sympy.simplify(params), theta

def cse_paramprint(cm):
    params, theta = sympyfied_predict(cm)
    full = sympy.Eq(
        lhs=sympy.symbols("loc scale df"),
        rhs=params,
    )
    def sgen():
        for s in range(0, 100):
            yield sympy.Symbol(f"s_{s}")

    replacements, reduced = sympy.cse(
        full,
        symbols=sgen()
    )
    
    latex = ""
    for lhs, rhs in replacements:
        v = sympy.Eq(lhs, rhs)
        latex += f"\n{v._repr_latex_()}"

    for eq in reduced:
        for lhs, rhs in zip(eq.lhs, eq.rhs):
            v = sympy.Eq(lhs, rhs)
            latex += f"\n{v._repr_latex_()}"
        
    # substitute unnecessary subexpressions
    lines = latex.split("\n")
    
    for l in reversed(range(0, len(lines))):
        line = lines[l]
        if line.startswith(r"$\displaystyle loc ="):
            lines.pop(l)
            pattern = line.split(" = ")[1].strip("$")
            for l, lold in enumerate(lines[:l+1]):
                lines[l] = lold.replace(pattern, "\mathrm{loc}")
    latex = "\n".join(lines)
    
    # Add parameter supports
    latex += "\n" + ",".join([p._repr_latex_() for p in theta[:-1]]) + r" \in \mathbb{R}"
    latex += "\n" + theta[-1]._repr_latex_() + r" \in \mathbb{R} > 0"
    
    for pattern, text in [
        ("scale", "\mathrm{scale}"),
        (r"} \mathrm", r"} \cdot \mathrm"),
        ("df", r"\nu"),
        ("\displaystyle", ""),
        (r"\frac{\log{\left(x \right)}}{\log{\left(10 \right)}}", "log_{10}(x)"),
    ]:
        latex = latex.replace(pattern, text)
        
    for line in latex.replace("$", "").replace("=", "&=").replace("\in", "&\in").split("\n"):
        line = line.strip()
        if line:
            print(line + r" \\")
    for line in latex.split("\n"):
        display(Latex(line))
    return

In [4]:
cse_paramprint(cm_glucose_linear)

\mathrm{loc} &= \mu_{0} + \mu_{1} x \\
\mathrm{scale} &= \mathrm{loc} \cdot \mathrm{scale}_{1} + \mathrm{scale}_{0} \\
\nu &= \nu \\
\mu_{0}, \mu_{1}, \mathrm{scale}_{0}, \mathrm{scale}_{1} &\in \mathbb{R} \\
\nu &\in \mathbb{R} > 0 \\


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [5]:
cse_paramprint(cm_glucose)

s_{0} &= - L_{L} + L_{U} \\
s_{1} &= e^{c} + 1 \\
s_{2} &= e^{- c} \\
s_{3} &= s_{1}^{s_{1} s_{2}} \\
\mathrm{loc} &= L_{L} + s_{0} \left(e^{s_{3} \left(\frac{S \left(I_{x} - x\right)}{s_{0}} + \frac{c}{s_{3}}\right)} + 1\right)^{- s_{2}} \\
\mathrm{scale} &= \mathrm{loc} \cdot \mathrm{scale}_{1} + \mathrm{scale}_{0} \\
\nu &= \nu \\
L_{L}, L_{U}, I_{x}, S, c, \mathrm{scale}_{0}, \mathrm{scale}_{1} &\in \mathbb{R} \\
\nu &\in \mathbb{R} > 0 \\


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [6]:
['L_L', 'L_U', 'log_I_x', 'S', 'c', 'scale_0', 'scale_1', 'df']
cm_biomass.theta_names = ['L_L', 'L_U', 'I_{log10(x)}', 'S_log10', 'c', 'scale_0', 'scale_1', 'df']
cse_paramprint(cm_biomass)

s_{0} &= - L_{L} + L_{U} \\
s_{1} &= e^{c} + 1 \\
s_{2} &= e^{- c} \\
s_{3} &= s_{1}^{s_{1} s_{2}} \\
\mathrm{loc} &= L_{L} + s_{0} \left(e^{s_{3} \left(\frac{S_{log10} \left(I_{log10(x)} - log_{10}(x)\right)}{s_{0}} + \frac{c}{s_{3}}\right)} + 1\right)^{- s_{2}} \\
\mathrm{scale} &= \mathrm{loc} \cdot \mathrm{scale}_{1} + \mathrm{scale}_{0} \\
\nu &= \nu \\
L_{L}, L_{U}, I_{log10(x)}, S_{log10}, c, \mathrm{scale}_{0}, \mathrm{scale}_{1} &\in \mathbb{R} \\
\nu &\in \mathbb{R} > 0 \\


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [7]:
%load_ext watermark
%watermark

Last updated: 2021-11-25T14:36:53.660108+01:00

Python implementation: CPython
Python version       : 3.7.9
IPython version      : 7.24.1

Compiler    : MSC v.1916 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 158 Stepping 10, GenuineIntel
CPU cores   : 6
Architecture: 64bit

